In [1]:
## conda environment Weather_Prediction

import os
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
pd.set_option('display.max_columns', None)

In [2]:
## get the variables

PATH_1=os.path.join('..','..','..','Weather_Data','ERA5','2004','era_2004.nc')
humidity_data = xr.open_dataset(PATH_1) ## variables
humidity_data

<xarray.Dataset> Size: 544MB
Dimensions:         (valid_time: 2928, pressure_level: 1, latitude: 100,
                     longitude: 232)
Coordinates:
    number          int64 8B ...
  * valid_time      (valid_time) datetime64[ns] 23kB 2004-01-01 ... 2004-12-3...
  * pressure_level  (pressure_level) float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
    expver          (valid_time) <U4 47kB ...
Data variables:
    r               (valid_time, pressure_level, latitude, longitude) float32 272MB ...
    q               (valid_time, pressure_level, latitude, longitude) float32 272MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T21:55 GRIB to CDM+CF via cfgrib-0.9.1...

In [3]:
humidity_data= humidity_data.drop_vars('number') ## not sure what these two are
humidity_data= humidity_data.drop_vars('expver')
humidity_data

<xarray.Dataset> Size: 543MB
Dimensions:         (valid_time: 2928, pressure_level: 1, latitude: 100,
                     longitude: 232)
Coordinates:
  * valid_time      (valid_time) datetime64[ns] 23kB 2004-01-01 ... 2004-12-3...
  * pressure_level  (pressure_level) float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
Data variables:
    r               (valid_time, pressure_level, latitude, longitude) float32 272MB ...
    q               (valid_time, pressure_level, latitude, longitude) float32 272MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T21:55 GRIB to CDM+CF via cfgrib-0.9.1...

In [4]:
humidity_data.to_dataframe()

r         q
valid_time          pressure_level latitude longitude                      
2004-01-01 00:00:00 1000.0         49.25    -124.85     99.344437  0.004035
                                            -124.60     98.680374  0.004095
                                            -124.35     99.545609  0.004173
                                            -124.10    100.238968  0.004214
                                            -123.85     96.621780  0.004246
...                                                           ...       ...
2004-12-31 21:00:00 1000.0         24.50    -68.10      63.329735  0.009759
                                            -67.85      64.634422  0.009921
                                            -67.60      65.626610  0.010060
                                            -67.35      66.193016  0.010150
                                            -67.10      66.523094  0.010213

[67929600 rows x 2 columns]

In [5]:
## resample the data to 1 year average

year_avg = humidity_data.resample(valid_time='1YE').mean(dim='valid_time')
year_avg

<xarray.Dataset> Size: 188kB
Dimensions:         (valid_time: 1, pressure_level: 1, latitude: 100,
                     longitude: 232)
Coordinates:
  * pressure_level  (pressure_level) float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
  * valid_time      (valid_time) datetime64[ns] 8B 2004-12-31
Data variables:
    r               (valid_time, pressure_level, latitude, longitude) float32 93kB ...
    q               (valid_time, pressure_level, latitude, longitude) float32 93kB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T21:55 GRIB to CDM+CF via cfgrib-0.9.1...

In [6]:
year_avg.to_dataframe()

r         q
valid_time pressure_level latitude longitude                     
2004-12-31 1000.0         49.25    -124.85    82.399345  0.006388
                                   -124.60    79.903801  0.006399
                                   -124.35    77.592377  0.006284
                                   -124.10    75.229958  0.006174
                                   -123.85    72.318367  0.006172
...                                                 ...       ...
                          24.50    -68.10     76.936798  0.013826
                                   -67.85     76.861259  0.013816
                                   -67.60     76.812920  0.013807
                                   -67.35     76.781342  0.013798
                                   -67.10     76.771767  0.013788

[23200 rows x 2 columns]

In [7]:
year_avg = year_avg.squeeze(dim="pressure_level") ## drop the pressure level dimension
year_avg

<xarray.Dataset> Size: 188kB
Dimensions:         (valid_time: 1, latitude: 100, longitude: 232)
Coordinates:
    pressure_level  float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
  * valid_time      (valid_time) datetime64[ns] 8B 2004-12-31
Data variables:
    r               (valid_time, latitude, longitude) float32 93kB 82.4 ... 7...
    q               (valid_time, latitude, longitude) float32 93kB 0.006388 ....
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T21:55 GRIB to CDM+CF via cfgrib-0.9.1...

In [8]:
SHAPE_PATH=os.path.join('..','..','..','Shapefiles','county_shapefiles','2008_county_shapefile','tl_2008_us_county.shp')
county_gdf = gpd.read_file(SHAPE_PATH)
county_gdf

,STATEFP,COUNTYFP,COUNTYNS,CNTYIDFP,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,geometry
0,19,175,00465276,19175,Union,Union County,06,H1,G4020,None,None,None,A,"POLYGON ((-94.47051 40.97504, -94.47050 40.975..."
1,19,177,00465277,19177,Van Buren,Van Buren County,06,H1,G4020,None,None,None,A,"POLYGON ((-91.96059 40.90070, -91.95925 40.900..."
2,20,097,00485013,20097,Kiowa,Kiowa County,06,H1,G4020,None,None,None,A,"POLYGON ((-99.01471 37.67895, -99.01473 37.678..."
3,20,109,00485019,20109,Logan,Logan County,06,H1,G4020,None,None,None,A,"POLYGON ((-100.89562 39.13329, -100.89404 39.1..."
4,20,003,00484971,20003,Anderson,Anderson County,06,H1,G4020,None,None,None,A,"POLYGON ((-95.51741 38.20619, -95.51741 38.206..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3228,35,015,00936829,35015,Eddy,Eddy County,06,H1,G4020,None,16100,None,A,"POLYGON ((-104.09554 32.00000, -104.09586 32.0..."
3229,40,015,01101795,40015,Caddo,Caddo County,06,H1,G4020,None,None,None,A,"POLYGON ((-98.36984 35.55133, -98.36964 35.551..."
3230,40,025,01101800,40025,Cimarron,Cimarron County,06,H1,G4020,None,None,None,A,"POLYGON ((-102.14252 36.50032, -102.14380 36.5..."
3231,40,069,01101822,40069,Johnston,Johnston County,06,H1,G4020,None,None,None,A,"POLYGON ((-96.74415 34.17223, -96.74416 34.172..."


In [9]:
geometry_column=county_gdf['geometry']
geometry_column

0       POLYGON ((-94.47051 40.97504, -94.47050 40.975...
1       POLYGON ((-91.96059 40.90070, -91.95925 40.900...
2       POLYGON ((-99.01471 37.67895, -99.01473 37.678...
3       POLYGON ((-100.89562 39.13329, -100.89404 39.1...
4       POLYGON ((-95.51741 38.20619, -95.51741 38.206...
                              ...                        
3228    POLYGON ((-104.09554 32.00000, -104.09586 32.0...
3229    POLYGON ((-98.36984 35.55133, -98.36964 35.551...
3230    POLYGON ((-102.14252 36.50032, -102.14380 36.5...
3231    POLYGON ((-96.74415 34.17223, -96.74416 34.172...
3232    POLYGON ((-84.27514 39.28921, -84.27467 39.289...
Name: geometry, Length: 3233, dtype: geometry

In [10]:
lat_lon=geometry_column.get_coordinates(index_parts=True)
lat_lon

x          y
0    0    -94.470506  40.975043
     1    -94.470501  40.975551
     2    -94.470512  40.978164
     3    -94.470498  40.978804
     4    -94.470502  40.979333
...              ...        ...
3232 3894 -84.275637  39.289242
     3895 -84.275550  39.289236
     3896 -84.275516  39.289234
     3897 -84.275508  39.289233
     3898 -84.275143  39.289207

[7781581 rows x 2 columns]

In [11]:
def single_county(var):
    
    ''' Function to interpolate the values in a single county by taking a 100 or less latitude and longitude pair.  
        Args:
        --------
             var (int): The index of the county in the shapefile.

        Returns:
        --------
            county_df: Dataframe consisting the values of the variables interpolated in the county.
    ''' 
    longitude= lat_lon.loc[(var), 'x']  # get the longitude 
    extract_val=len(longitude)        ## find number of longitude that a shapfile has

    ## if number of longitude or latitude is less than 100, that many lat-lon pairs will be extracted, if not, a 100 values
    if extract_val < 100:
        extract_val=extract_val
    else:
        extract_val=100

    longitude=longitude[:extract_val]  ## extract first 100 values or values less than 100
    
    latitude= lat_lon.loc[(var), 'y']   # get the latitude values
    latitude=latitude[:extract_val]    ## extract first 100 values or values less than 100

    lat_list=latitude.tolist()
    lon_list=longitude.tolist()

    ## find the corresponding values of the variables in the finer grid

    year_avg_finer= year_avg.interp(longitude=lon_list, latitude=lat_list)
    year_avg_finer
    
    df = year_avg_finer.squeeze(dim="valid_time") ## remove the valid_time dimension, note that this will appear as a column
                                                  ## after removed

    ## get only the 100 (or less) pair of latitude and longitude from the lat and lon list from the 100,000 (or less) rows
    get_vals=[]

    for i in range(0, extract_val):
        row=df.isel(latitude=[i], longitude=[i])
        row_df=row.to_dataframe()
        get_vals.append(row_df)

    initial_df=pd.concat(get_vals)
    initial_df=initial_df.drop('valid_time', axis=1) ## remove the valid_time column 
    
    mean=initial_df.mean() ## find the average of the 100 (or less) values
    average_df = pd.DataFrame(mean).T 

    county_df= average_df.convert_dtypes() ## the dtype of the mean values will be object, so convert that to float
    county_df
    
    return county_df

In [12]:
%%time

## include all counties

df_list=[]

for i in range(0,3233): ## A total of 3233 counties. index in shape file starts from 0 and ends in 3232
    try:
        df_list.append(single_county(i))
    except:
        pass

CPU times: total: 6min 19s
Wall time: 6min 25s


In [13]:
concatenate_df=pd.concat(df_list)
final_df=concatenate_df.reset_index(drop=True)
final_df = final_df.drop('pressure_level', axis=1)
final_df

,r,q
0,70.060492,0.006914
1,69.274032,0.006945
2,62.11417,0.007206
3,59.90817,0.006406
4,67.773759,0.007829
...,...,...
3228,51.67126,0.006641
3229,65.086649,0.00832
3230,53.399826,0.006021
3231,67.311244,0.009361


In [14]:
## lets first create a fips column in the original shape file

## lets create the fips column in the dataframe which is the sum of state and county both as dtype string.
## Note: not a numerical sum
## This step is necessary because there can be several counties with idential names. 

county_gdf['fips']=county_gdf['STATEFP'] + county_gdf['COUNTYFP']
county_gdf.dtypes
## convert the data type
county_gdf['fips']=county_gdf['fips'].astype(str).astype(int)
county_gdf.dtypes

STATEFP       object
COUNTYFP      object
COUNTYNS      object
CNTYIDFP      object
NAME          object
NAMELSAD      object
LSAD          object
CLASSFP       object
MTFCC         object
CSAFP         object
CBSAFP        object
METDIVFP      object
FUNCSTAT      object
geometry    geometry
fips           int32
dtype: object

In [15]:
county_gdf = county_gdf.drop(['STATEFP','COUNTYFP','COUNTYNS','CNTYIDFP','NAMELSAD',
                              'LSAD','CLASSFP','MTFCC','CSAFP','CBSAFP','METDIVFP',
                             'FUNCSTAT'], axis=1)
county_gdf

,NAME,geometry,fips
0,Union,"POLYGON ((-94.47051 40.97504, -94.47050 40.975...",19175
1,Van Buren,"POLYGON ((-91.96059 40.90070, -91.95925 40.900...",19177
2,Kiowa,"POLYGON ((-99.01471 37.67895, -99.01473 37.678...",20097
3,Logan,"POLYGON ((-100.89562 39.13329, -100.89404 39.1...",20109
4,Anderson,"POLYGON ((-95.51741 38.20619, -95.51741 38.206...",20003
...,...,...,...
3228,Eddy,"POLYGON ((-104.09554 32.00000, -104.09586 32.0...",35015
3229,Caddo,"POLYGON ((-98.36984 35.55133, -98.36964 35.551...",40015
3230,Cimarron,"POLYGON ((-102.14252 36.50032, -102.14380 36.5...",40025
3231,Johnston,"POLYGON ((-96.74415 34.17223, -96.74416 34.172...",40069


In [16]:
merged_df=pd.merge(county_gdf, final_df, left_index=True, right_index=True)
merged_df

,NAME,geometry,fips,r,q
0,Union,"POLYGON ((-94.47051 40.97504, -94.47050 40.975...",19175,70.060492,0.006914
1,Van Buren,"POLYGON ((-91.96059 40.90070, -91.95925 40.900...",19177,69.274032,0.006945
2,Kiowa,"POLYGON ((-99.01471 37.67895, -99.01473 37.678...",20097,62.11417,0.007206
3,Logan,"POLYGON ((-100.89562 39.13329, -100.89404 39.1...",20109,59.90817,0.006406
4,Anderson,"POLYGON ((-95.51741 38.20619, -95.51741 38.206...",20003,67.773759,0.007829
...,...,...,...,...,...
3228,Eddy,"POLYGON ((-104.09554 32.00000, -104.09586 32.0...",35015,51.67126,0.006641
3229,Caddo,"POLYGON ((-98.36984 35.55133, -98.36964 35.551...",40015,65.086649,0.00832
3230,Cimarron,"POLYGON ((-102.14252 36.50032, -102.14380 36.5...",40025,53.399826,0.006021
3231,Johnston,"POLYGON ((-96.74415 34.17223, -96.74416 34.172...",40069,67.311244,0.009361


In [17]:
county_df=merged_df.dropna()
county_df

,NAME,geometry,fips,r,q
0,Union,"POLYGON ((-94.47051 40.97504, -94.47050 40.975...",19175,70.060492,0.006914
1,Van Buren,"POLYGON ((-91.96059 40.90070, -91.95925 40.900...",19177,69.274032,0.006945
2,Kiowa,"POLYGON ((-99.01471 37.67895, -99.01473 37.678...",20097,62.11417,0.007206
3,Logan,"POLYGON ((-100.89562 39.13329, -100.89404 39.1...",20109,59.90817,0.006406
4,Anderson,"POLYGON ((-95.51741 38.20619, -95.51741 38.206...",20003,67.773759,0.007829
...,...,...,...,...,...
3228,Eddy,"POLYGON ((-104.09554 32.00000, -104.09586 32.0...",35015,51.67126,0.006641
3229,Caddo,"POLYGON ((-98.36984 35.55133, -98.36964 35.551...",40015,65.086649,0.00832
3230,Cimarron,"POLYGON ((-102.14252 36.50032, -102.14380 36.5...",40025,53.399826,0.006021
3231,Johnston,"POLYGON ((-96.74415 34.17223, -96.74416 34.172...",40069,67.311244,0.009361


In [18]:
type(county_df)

geopandas.geodataframe.GeoDataFrame

In [19]:
## convert into pandas dataframe without the geometry column
pd.options.display.float_format = '{:.15f}'.format ## see 15 decimal places of the numbers
county_var=pd.DataFrame(county_df.drop(columns='geometry')) 
county_var

,NAME,fips,r,q
0,Union,19175,70.060491904822015,0.006914002964493
1,Van Buren,19177,69.274032194927329,0.006945161038423
2,Kiowa,20097,62.114170309936227,0.007205522174641
3,Logan,20109,59.908169524652045,0.006405842919948
4,Anderson,20003,67.773758591761620,0.007828580226915
...,...,...,...,...
3228,Eddy,35015,51.671260142652983,0.006640569166983
3229,Caddo,40015,65.086648918098462,0.008319654552725
3230,Cimarron,40025,53.399825570375739,0.006021495285186
3231,Johnston,40069,67.311244073649888,0.009360751952113


In [20]:
county_var = county_var.sort_values(by='fips')
county_var

,NAME,fips,r,q
1536,Autauga,1001,66.558014063306331,0.009697563780323
441,Baldwin,1003,73.369163583912311,0.011656178365907
2221,Barbour,1005,67.814059080496577,0.010077728872073
2388,Bibb,1007,69.742225785496402,0.009727866455881
2153,Blount,1009,70.578745524517274,0.009438419178427
...,...,...,...,...
1193,Sweetwater,56037,59.970412415279824,0.003786306106378
1865,Teton,56039,68.288928190678590,0.003408541038001
2348,Uinta,56041,62.111659824776844,0.003554728920947
2349,Washakie,56043,53.438291845503052,0.003987152800384


In [21]:
county_var.to_pickle('era_2004.pkl')